# MLE

In [3]:
using Optim,NLSolversBase,Random
using LinearAlgebra: diag
Random.seed!(0); # Fix random seed generator for reproducibility

In [4]:
n = 500                             # Number of observations
nvar = 2


# Number of variables
β = ones(nvar) * 3.0                # True coefficients
x = [ones(n) randn(n, nvar - 1)]    # X matrix of explanatory variables plus constant
ε = randn(n) * 0.5                  # Error variance
y = x * β + ε;                      # Generate Data

In [5]:
function Log_Likelihood(X, Y, β, log_σ)
    σ = exp(log_σ)
    llike = -n/2*log(2π) - n/2* log(σ^2) - (sum((Y - X * β).^2) / (2σ^2))
    llike = -llike
end

Log_Likelihood (generic function with 1 method)

`TwiceDifferentiable`

In [6]:
func = TwiceDifferentiable(vars -> Log_Likelihood(x, y, vars[1:nvar], vars[nvar + 1]),
                           ones(nvar+1); autodiff=:forward);

The second input specifies the starting values.

In [8]:
opt = optimize(func, ones(nvar+1))

 * Status: success

 * Candidate solution
    Minimizer: [3.00e+00, 2.96e+00, -6.49e-01]
    Minimum:   3.851229e+02

 * Found with
    Algorithm:     Newton's Method
    Initial Point: [1.00e+00, 1.00e+00, 1.00e+00]

 * Convergence measures
    |x - x'|               = 2.62e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.72e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.12e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.33e-15 ≰ 0.0e+00
    |g(x)|                 = 3.41e-13 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    7
    f(x) calls:    31
    ∇f(x) calls:   31
    ∇²f(x) calls:  7


In [9]:
parameters=Optim.minimizer(opt)

3-element Array{Float64,1}:
  3.002788633849947
  2.964549617572727
 -0.648692780562844

In [10]:
parameters[nvar+1] = exp(parameters[nvar+1])

0.5227286513837306

In [11]:
numerical_hessian=hessian!(func,parameters)

3×3 Array{Float64,2}:
 175.766        -12.0877        5.67464e-14
 -12.0877       182.437         5.88344e-15
   5.67464e-14    5.88344e-15  96.0542

In [12]:
var_cov_matrix=inv(numerical_hessian)

3×3 Array{Float64,2}:
  0.00571544    0.000378687  -3.39973e-18
  0.000378687   0.00550643   -5.60994e-19
 -3.39973e-18  -5.60994e-19   0.0104108

In [13]:
β=parameters[1:nvar]

2-element Array{Float64,1}:
 3.002788633849947
 2.964549617572727

In [15]:
temp=diag(var_cov_matrix)
temp1=temp[1:nvar]

2-element Array{Float64,1}:
 0.005715441191953887
 0.005506430533329349

In [16]:
t_stats=β./sqrt.(temp1)

2-element Array{Float64,1}:
 39.719144251161474
 39.95063081460274